In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder

# data preprocessing

In [10]:
data_train = pd.read_csv("../data/for_train/train_final.csv")
data_train.head()
data_train.shape

(34613, 1236)

In [11]:
data_train.drop(['Unnamed: 0'], axis=1, inplace=True)

In [12]:
dl0 = pd.read_csv("data/for_train/df_league_0.csv")
dl1 = pd.read_csv("data/for_train/df_league_1.csv")
dl2 = pd.read_csv("data/for_train/df_league_2.csv")
dl3 = pd.read_csv("data/for_train/df_league_3.csv")

country_england = data_train[data_train['Country_england'] == 1]

In [13]:
country_england = data_train[data_train['Country_england'] == 1]
country_france = data_train[data_train['Country_france'] == 1]

In [14]:
train_split = data_train[data_train['Season'] != 2122]
test_split = data_train[data_train['Season'] == 2122]
print(train_split.shape)
print(test_split.shape)

(27447, 1235)
(7166, 1235)


In [15]:
subsets = {category: subset for category, subset in data_train.groupby('League')}

for category, subset in subsets.items():
    globals()[f'dl{category}'] = subset

In [16]:
dl0.head()

,League,Season,FTR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,...,Referee_T Kettle,Referee_T Nield,Referee_T Reeves,Referee_T Robinson,Referee_Unknown,Referee_W Anderson,Referee_W Collum,Referee_W Finnie,Referee_k Stroud,Referee_l Wilson
1297,0,1718,H,1.53,4.5,6.50,1.50,4.60,6.75,1.47,...,False,False,False,False,False,False,False,False,False,False
1298,0,1718,A,11.00,5.5,1.33,11.00,5.25,1.30,8.00,...,False,False,False,False,False,False,False,False,False,False
1299,0,1718,A,1.83,3.6,5.00,1.80,3.50,4.75,1.85,...,False,False,False,False,False,False,False,False,False,False
1300,0,1718,A,1.25,6.5,15.00,1.22,6.50,12.50,1.22,...,False,False,False,False,False,False,False,False,False,False
1301,0,1718,H,1.70,3.8,5.75,1.70,3.60,5.50,1.70,...,False,False,False,False,False,False,False,False,False,False


In [17]:
label_encoder = LabelEncoder()

data_train['FTR'] = label_encoder.fit_transform(data_train['FTR'])

print(data_train['FTR'].value_counts())


2    14952
0    10542
1     9119
Name: FTR, dtype: int64


# models by league

In [35]:
scaler = StandardScaler()
def pipeline1(model, df):
    X = df.drop('FTR', axis=1)
    y = df['FTR']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred, average='macro'))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    
def pipeline2(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("F1 Score:", f1_score(y_test, y_pred, average='macro'))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [19]:
model_rf = RandomForestClassifier()
pipeline1(model_rf, dl0)

Accuracy: 0.5187713310580204
F1 Score: 0.5137105753924016
[[145  96  51]
 [ 31 115  53]
 [ 70 122 196]]
              precision    recall  f1-score   support

           A       0.59      0.50      0.54       292
           D       0.35      0.58      0.43       199
           H       0.65      0.51      0.57       388

    accuracy                           0.52       879
   macro avg       0.53      0.53      0.51       879
weighted avg       0.56      0.52      0.53       879



In [20]:
pipeline1(model_rf, dl1)

Accuracy: 0.5756310495350161
F1 Score: 0.5393112489588429
[[ 852   79  721]
 [ 324  422  612]
 [ 429   71 1759]]
              precision    recall  f1-score   support

           A       0.53      0.52      0.52      1652
           D       0.74      0.31      0.44      1358
           H       0.57      0.78      0.66      2259

    accuracy                           0.58      5269
   macro avg       0.61      0.54      0.54      5269
weighted avg       0.60      0.58      0.56      5269



In [21]:
pipeline1(model_rf, dl2)

Accuracy: 0.5275712515489467
F1 Score: 0.5022371929637189
[[386  83 449]
 [192 342 399]
 [287 115 975]]
              precision    recall  f1-score   support

           A       0.45      0.42      0.43       918
           D       0.63      0.37      0.46       933
           H       0.53      0.71      0.61      1377

    accuracy                           0.53      3228
   macro avg       0.54      0.50      0.50      3228
weighted avg       0.54      0.53      0.52      3228



In [22]:
pipeline1(model_rf, dl3)

Accuracy: 0.4188118811881188
F1 Score: 0.3997052805937192
[[117 102  98]
 [ 69  78 115]
 [ 87 116 228]]
              precision    recall  f1-score   support

           A       0.43      0.37      0.40       317
           D       0.26      0.30      0.28       262
           H       0.52      0.53      0.52       431

    accuracy                           0.42      1010
   macro avg       0.40      0.40      0.40      1010
weighted avg       0.42      0.42      0.42      1010



In [23]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs') 
pipeline1(model_lr, dl0)

Accuracy: 0.5233219567690558
F1 Score: 0.47253271804860075
[[161  51  80]
 [ 69  44  86]
 [ 91  42 255]]
              precision    recall  f1-score   support

           A       0.50      0.55      0.53       292
           D       0.32      0.22      0.26       199
           H       0.61      0.66      0.63       388

    accuracy                           0.52       879
   macro avg       0.48      0.48      0.47       879
weighted avg       0.51      0.52      0.51       879



/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
pipeline1(model_lr, dl1)

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.48529132662744356
F1 Score: 0.4285807499608502
[[ 760  225  667]
 [ 417  220  721]
 [ 395  287 1577]]
              precision    recall  f1-score   support

           A       0.48      0.46      0.47      1652
           D       0.30      0.16      0.21      1358
           H       0.53      0.70      0.60      2259

    accuracy                           0.49      5269
   macro avg       0.44      0.44      0.43      5269
weighted avg       0.46      0.49      0.46      5269



In [25]:
pipeline1(model_lr, dl2)

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.45322180916976457
F1 Score: 0.42920174404489525
[[328 204 386]
 [222 309 402]
 [271 280 826]]
              precision    recall  f1-score   support

           A       0.40      0.36      0.38       918
           D       0.39      0.33      0.36       933
           H       0.51      0.60      0.55      1377

    accuracy                           0.45      3228
   macro avg       0.43      0.43      0.43      3228
weighted avg       0.44      0.45      0.45      3228



In [26]:
pipeline1(model_lr, dl3)

Accuracy: 0.41287128712871285
F1 Score: 0.3798310249843209
[[115  66 136]
 [ 79  56 127]
 [ 93  92 246]]
              precision    recall  f1-score   support

           A       0.40      0.36      0.38       317
           D       0.26      0.21      0.24       262
           H       0.48      0.57      0.52       431

    accuracy                           0.41      1010
   macro avg       0.38      0.38      0.38      1010
weighted avg       0.40      0.41      0.40      1010



/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
# Create a Decision Tree classifier
model_DT = DecisionTreeClassifier()
pipeline1(model_DT, dl0)

Accuracy: 0.42662116040955633
F1 Score: 0.4182639010547895
[[ 77 156  59]
 [ 13 164  22]
 [ 49 205 134]]
              precision    recall  f1-score   support

           A       0.55      0.26      0.36       292
           D       0.31      0.82      0.45       199
           H       0.62      0.35      0.44       388

    accuracy                           0.43       879
   macro avg       0.50      0.48      0.42       879
weighted avg       0.53      0.43      0.42       879



In [28]:
pipeline1(model_DT, dl1)

Accuracy: 0.5593091668248245
F1 Score: 0.5562374670088396
[[ 742  209  701]
 [ 207  812  339]
 [ 568  298 1393]]
              precision    recall  f1-score   support

           A       0.49      0.45      0.47      1652
           D       0.62      0.60      0.61      1358
           H       0.57      0.62      0.59      2259

    accuracy                           0.56      5269
   macro avg       0.56      0.55      0.56      5269
weighted avg       0.56      0.56      0.56      5269



In [29]:
pipeline1(model_DT, dl2)

Accuracy: 0.5161090458488228
F1 Score: 0.514820747241558
[[366 122 430]
 [128 589 216]
 [433 233 711]]
              precision    recall  f1-score   support

           A       0.39      0.40      0.40       918
           D       0.62      0.63      0.63       933
           H       0.52      0.52      0.52      1377

    accuracy                           0.52      3228
   macro avg       0.51      0.52      0.51      3228
weighted avg       0.52      0.52      0.52      3228



In [30]:
pipeline1(model_DT, dl3)

Accuracy: 0.30396039603960395
F1 Score: 0.30312032943611894
[[ 84 161  72]
 [127  84  51]
 [ 96 196 139]]
              precision    recall  f1-score   support

           A       0.27      0.26      0.27       317
           D       0.19      0.32      0.24       262
           H       0.53      0.32      0.40       431

    accuracy                           0.30      1010
   macro avg       0.33      0.30      0.30      1010
weighted avg       0.36      0.30      0.32      1010



# main models

In [31]:
model1 = RandomForestClassifier()
pipeline1(model1, data_train)

Accuracy: 0.5676040061633282
F1 Score: 0.5362875274565945
[[1527  119 1523]
 [ 612  922 1239]
 [ 860  137 3445]]
              precision    recall  f1-score   support

           0       0.51      0.48      0.50      3169
           1       0.78      0.33      0.47      2773
           2       0.56      0.78      0.65      4442

    accuracy                           0.57     10384
   macro avg       0.62      0.53      0.54     10384
weighted avg       0.60      0.57      0.55     10384



In [32]:
model2 = LogisticRegression(multi_class='multinomial', solver='lbfgs') 
# 'lbfgs' solver supports the multinomial option
pipeline1(model2, data_train)

Accuracy: 0.47804314329738057
F1 Score: 0.42394718035788975
[[1413  478 1278]
 [ 795  461 1517]
 [ 814  538 3090]]
              precision    recall  f1-score   support

           0       0.47      0.45      0.46      3169
           1       0.31      0.17      0.22      2773
           2       0.53      0.70      0.60      4442

    accuracy                           0.48     10384
   macro avg       0.43      0.44      0.42     10384
weighted avg       0.45      0.48      0.45     10384



/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
model4 = DecisionTreeClassifier()
pipeline1(model4, data_train)  

Accuracy: 0.5600924499229584
F1 Score: 0.5574092462153333
[[1397  423 1349]
 [ 438 1734  601]
 [1157  600 2685]]
              precision    recall  f1-score   support

           0       0.47      0.44      0.45      3169
           1       0.63      0.63      0.63      2773
           2       0.58      0.60      0.59      4442

    accuracy                           0.56     10384
   macro avg       0.56      0.56      0.56     10384
weighted avg       0.56      0.56      0.56     10384



TypeError: 'method' object is not subscriptable

from sklearn import svm
model3 = svm.SVC()
pipeline1(model3, data_train)  

In [39]:
pipeline2(model4, train_split.drop('FTR', axis=1), test_split.drop('FTR', axis=1) , train_split['FTR'], test_split['FTR'])

Accuracy: 0.5308400781468043
F1 Score: 0.5323174071434978
[[1055  232  932]
 [ 359 1028  517]
 [ 967  355 1721]]
              precision    recall  f1-score   support

           A       0.44      0.48      0.46      2219
           D       0.64      0.54      0.58      1904
           H       0.54      0.57      0.55      3043

    accuracy                           0.53      7166
   macro avg       0.54      0.53      0.53      7166
weighted avg       0.54      0.53      0.53      7166



In [ ]:
from sklearn import svm
model3 = svm.SVC()
pipeline1(model3, data_train)  

In [45]:
def tune_parameters(model, param_grid, df, cv):
    X = df.drop('FTR', axis=1)
    y = df['FTR']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    clf= GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1_macro',
                        cv=cv, n_jobs=-1, verbose=2)
    clf.fit(X_train, y_train)
    best_parameters = clf.best_params_
    return best_parameters

def tune_parameters_custom(model, param_grid, X_train, X_test, y_train, y_test, cv):
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    clf= GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1_macro',
                        cv=cv, n_jobs=-1, verbose=2)
    clf.fit(X_train, y_train)
    best_parameters = clf.best_params_
    return best_parameters

In [46]:
best_parameters_model4_season = tune_parameters_custom(model=model4,param_grid=param_grid,
                                                X_train =train_split.drop('FTR', axis=1), 
                                                X_test = test_split.drop('FTR', axis=1) ,
                                                y_train = train_split['FTR'], 
                                                y_test = test_split['FTR'] ,
                                                cv = 5)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=  12.6s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=  13.3s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=  13.3s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=  13.3s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=  13.4s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=  13.5s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=  13.6s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=2; total time=  13.6s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1, min_samples_split=5; total time=   5.9s
[CV] END criterion=gini, max_depth=3, min_samples_leaf=1

In [40]:
param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}
best_parameters_model4 = tune_parameters(model=model4,param_grid=param_grid, df = data_train,cv = 5)

NameError: name 'tune_parameters' is not defined

In [49]:
print("Best Parameters:", best_parameters_model4_season )

Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [51]:

model4_tuned = DecisionTreeClassifier(max_depth=best_parameters_model4['max_depth'], 
                                min_samples_split=best_parameters_model4['min_samples_split'], 
                                min_samples_leaf=best_parameters_model4['min_samples_leaf'], 
                                criterion=best_parameters_model4['criterion'])
pipeline1(model4_tuned, data_train)

NameError: name 'best_parameters_model4' is not defined

In [52]:

model4_tuned_custom = DecisionTreeClassifier(max_depth=best_parameters_model4_season['max_depth'], 
                                min_samples_split=best_parameters_model4_season['min_samples_split'], 
                                min_samples_leaf=best_parameters_model4_season['min_samples_leaf'], 
                                criterion=best_parameters_model4_season['criterion'])
pipeline2(model4_tuned_custom, train_split.drop('FTR', axis=1), test_split.drop('FTR', axis=1) , train_split['FTR'], test_split['FTR'])

Accuracy: 0.62977951437343
F1 Score: 0.6234538137821878
[[1049  340  830]
 [ 155 1366  383]
 [ 544  401 2098]]
              precision    recall  f1-score   support

           A       0.60      0.47      0.53      2219
           D       0.65      0.72      0.68      1904
           H       0.63      0.69      0.66      3043

    accuracy                           0.63      7166
   macro avg       0.63      0.63      0.62      7166
weighted avg       0.63      0.63      0.63      7166



In [ ]:
param_grid = {
    'C': np.logspace(-4, 4, 20),          
    'penalty': ['l2'],             
    'solver': ['lbfgs']               
}

best_parameters_model2 = tune_parameters(model=model2,param_grid=param_grid, df = data_train, cv = 5)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .................C=0.0001, penalty=l2, solver=lbfgs; total time=   8.0s
[CV] END .................C=0.0001, penalty=l2, solver=lbfgs; total time=   8.1s
[CV] END .................C=0.0001, penalty=l2, solver=lbfgs; total time=   8.6s
[CV] END .................C=0.0001, penalty=l2, solver=lbfgs; total time=   9.5s
[CV] END .................C=0.0001, penalty=l2, solver=lbfgs; total time=   9.6s
[CV] END .C=0.00026366508987303583, penalty=l2, solver=lbfgs; total time=   9.7s
[CV] END .C=0.00026366508987303583, penalty=l2, solver=lbfgs; total time=   9.9s
[CV] END .C=0.00026366508987303583, penalty=l2, solver=lbfgs; total time=  10.3s
[CV] END .C=0.00026366508987303583, penalty=l2, solver=lbfgs; total time=   7.8s
[CV] END .C=0.00026366508987303583, penalty=l2, solver=lbfgs; total time=   8.7s
[CV] END ..C=0.0006951927961775605, penalty=l2, solver=lbfgs; total time=  10.1s
[CV] END ..C=0.0006951927961775605, penalty=l2,

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ...C=0.012742749857031334, penalty=l2, solver=lbfgs; total time=  31.8s


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ...C=0.012742749857031334, penalty=l2, solver=lbfgs; total time=  29.7s


...

/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................C=10000.0, penalty=l2, solver=lbfgs; total time=  18.7s


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END ................C=10000.0, penalty=l2, solver=lbfgs; total time=  17.3s


/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
model2_tuned = LogisticRegression(solver=best_parameters_model2['solver'],
                                    penalty = best_parameters_model2['penalty'],
                                    C=best_parameters_model2['C'])

pipeline1(model2_tuned, data_train)

Accuracy: 0.4774653312788906
F1 Score: 0.4237882320766317
[[1406  483 1280]
 [ 796  466 1511]
 [ 813  543 3086]]
              precision    recall  f1-score   support

           0       0.47      0.44      0.45      3169
           1       0.31      0.17      0.22      2773
           2       0.53      0.69      0.60      4442

    accuracy                           0.48     10384
   macro avg       0.43      0.44      0.42     10384
weighted avg       0.45      0.48      0.45     10384



/Users/anastasiiahoncharuk/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
